In [14]:
# normal imports
import time
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from datetime import datetime
from decimal import Decimal

# nautilus trader imports
from nautilus_trader.backtest.engine import BacktestEngine, BacktestEngineConfig
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.enums import AccountType, OmsType
from nautilus_trader.model.identifiers import Venue, ClientId
from nautilus_trader.model.objects import Money
from nautilus_trader.persistence.wranglers import BarDataWrangler, TradeTickDataWrangler
from nautilus_trader.test_kit.providers import TestInstrumentProvider, TestDataProvider
from nautilus_trader.config import StrategyConfig
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.instruments import Instrument
from nautilus_trader.model.orders.list import OrderList
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model.data import Bar, BarSpecification, BarType
from nautilus_trader.model.enums import OrderSide, PositionSide, TimeInForce
from nautilus_trader.persistence.wranglers import BarDataWrangler
from BasicMRStrategy import BasicMR, BasicMRConfig
from nautilus_trader.backtest.node import BacktestNode, BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig,  ImportableActorConfig, StreamingConfig
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.model.enums import AggregationSource
from nautilus_trader.core.datetime import dt_to_unix_nanos
from util import yf_to_timeseries
from nautilus_trader.core.data import Data

from itertools import repeat
from nautilus_trader.model.objects import Price

# other file related imports
from pathlib import Path
import fsspec 
import shutil

[*********************100%***********************]  1 of 1 completed


In [ ]:
# creating instrument
MSFT_SIM = TestInstrumentProvider.equity(symbol="MSFT", venue="SIM")

# downloading data
start_str = "2023-01-01"
end_str = "2023-01-31"

msft_df = yf.download("MSFT", start=start_str, end=end_str, interval="1h")
msft_ts = yf_to_timeseries(msft_df, 7)


# processing data
bartype = BarType.from_str("MSFT.SIM-1-HOUR-LAST-EXTERNAL")

In [16]:
ts_event = msft_ts.index.view(np.uint64)
ts_init = ts_event.copy()

In [18]:
msft_ts.rename(columns={'Price': 'price'}, inplace=True)
msft_ts["quantity"] = np.ones(len(msft_ts))
msft_ts["trade_id"] = np.arange(len(msft_ts))

wrangler = TradeTickDataWrangler(instrument=MSFT_SIM)

In [23]:
ticks = wrangler.process(msft_ts, ts_init_delta=0)

In [25]:
engine = BacktestEngine()

SIM = Venue("SIM")
engine.add_venue(
    venue=SIM,
    oms_type=OmsType.HEDGING,
    account_type=AccountType.CASH,
    starting_balances=[Money(1_000_000, USD)]
)

In [26]:
engine.add_instrument(MSFT_SIM)
engine.add_data(ticks)

In [28]:
from BasicMRStrategy import BasicMR, BasicMRConfig

strategy_config = BasicMRConfig(
    instrument_id=MSFT_SIM.id,
    bar_type=bartype,
    trade_size=Decimal(1)
)
strategy = BasicMR(strategy_config)
engine.add_strategy(strategy)

In [32]:
engine.run()

""


In [39]:
# venue config
venues = [
    BacktestVenueConfig(
        name="SIM",
        oms_type="NETTING",
        account_type="CASH",
        base_currency="USD",
        starting_balances=["1_000_000 USD"],
    ),
]


start = dt_to_unix_nanos(pd.Timestamp(start_str, tz="EST"))
end =  dt_to_unix_nanos(pd.Timestamp(end_str, tz="EST"))

# data config
data = [
    BacktestDataConfig(
        catalog_path=str(CATALOG_PATH),
        data_cls="bar",
        instrument_id=instrument.id,
        start_time=start,
        end_time=end,
    ),
]
# strategy 
strategy = ImportableStrategyConfig(
        strategy_path="BasicMRStrategy:BasicMR",
        config_path="BasicMRStrategy:BasicMRConfig",
        config=dict(
            instrument_id=instrument.id,
            bar_type=bartype,
            trade_size=Decimal(10_000),
        ),
    )

# create engine config
engine = BacktestEngineConfig(
        trader_id="BACKTESTER-001",
        strategies=[strategy])


# backtest config
run_config = BacktestRunConfig(engine=engine, venues=venues, data=data)
node = BacktestNode(configs=[run_config])

results = node.run()